### Visión por Computador

**Grupo 17**:

- Sara Expósito Suárez
- Alejandro Padrón Ossorio

## Detección y caracterización de caras

### Paquetes necesarios

In [ ]:
import cv2
import numpy as np
import mediapipe as mp
from PIL import Image, ImageSequence

### Rainbow Puke

In [ ]:
# Inicializar Mediapipe para detección de landmarks faciales
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, min_detection_confidence=0.5)

# Cargar el GIF y convertirlo a una lista de fotogramas
gif = Image.open("rainbow.gif")
gif_frames = [frame.copy() for frame in ImageSequence.Iterator(gif)]

# Ajusta la función gif_overlay para cambiar la posición del GIF
def gif_overlay(img, frame_position, scale_width):
    """Superpone el GIF escalado en la posición especificada de la imagen."""
    
    # Seleccionar el cuadro actual del GIF
    frame = gif_frames[int(cv2.getTickCount() % len(gif_frames)) % len(gif_frames)]
    
    # Redimensionar el GIF al tamaño adecuado
    frame = frame.resize((scale_width, int(scale_width * frame.size[1] / frame.size[0])))
    frame = frame.convert("RGBA")
    frame_np = np.array(frame)
    
    # Coordenadas y dimensiones
    mouth_center_x, mouth_center_y = frame_position
    gif_height, gif_width = frame_np.shape[:2]

    # Ajustar la posición de y para que el borde superior esté en el centro de la boca
    top_left_x = mouth_center_x - gif_width // 2
    top_left_y = mouth_center_y

    # Verificar límites de la imagen para no salirnos
    if top_left_y + gif_height > img.shape[0]:
        gif_height = img.shape[0] - top_left_y
        frame_np = frame_np[:gif_height, :, :]
    if top_left_x + gif_width > img.shape[1]:
        gif_width = img.shape[1] - top_left_x
        frame_np = frame_np[:, :gif_width, :]

    # Superponer el GIF en la imagen
    for c in range(3):  # Canales RGB
        img[top_left_y:top_left_y+gif_height, top_left_x:top_left_x+gif_width, c] = \
            frame_np[:gif_height, :gif_width, c] * (frame_np[:gif_height, :gif_width, 3] / 255.0) + \
            img[top_left_y:top_left_y+gif_height, top_left_x:top_left_x+gif_width, c] * \
            (1.0 - frame_np[:gif_height, :gif_width, 3] / 255.0)

def mouth_open(landmarks, threshold=0.05):
    """Calcula si la boca está abierta basado en la distancia entre puntos específicos."""
    top_lip = landmarks[13]  # Landmark del labio superior
    bottom_lip = landmarks[14]  # Landmark del labio inferior
    mouth_height = abs(bottom_lip.y - top_lip.y)
    return mouth_height > threshold

# Inicializar captura de video
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.flip(frame, 1)  # Voltear la imagen para mirarse en el espejo
    
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Obtener coordenadas de los landmarks de la boca
            mouth_top = face_landmarks.landmark[13]  # Labio superior
            mouth_bottom = face_landmarks.landmark[14]  # Labio inferior
            mouth_left = face_landmarks.landmark[78]  # Lado izquierdo de la boca
            mouth_right = face_landmarks.landmark[308]  # Lado derecho de la boca
            
            # Calcular posición y ancho de la boca
            mouth_center_x = int((mouth_top.x + mouth_bottom.x) / 2 * frame.shape[1])
            mouth_center_y = int((mouth_top.y + mouth_bottom.y) / 2 * frame.shape[0])
            mouth_width = int(abs(mouth_right.x - mouth_left.x) * frame.shape[1])

            # Verificar si la boca está abierta
            if mouth_open(face_landmarks.landmark):
                gif_overlay(frame, (mouth_center_x, mouth_center_y), mouth_width)

    cv2.imshow("Mouth Open Detection with GIF", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


### Sparkling Wink

In [ ]:
# Configuración de MediaPipe
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True)
mp_drawing = mp.solutions.drawing_utils

# Cargar la imagen de la estrella y convertirla a RGBA
star_img = Image.open("sparkles.png").convert("RGBA")

# Función para calcular la distancia vertical entre los párpados de un ojo
def eye_aspect_ratio(eye_landmarks):
    vertical_dist = np.linalg.norm(np.array([eye_landmarks[1].x, eye_landmarks[1].y]) - 
                                   np.array([eye_landmarks[5].x, eye_landmarks[5].y]))
    horizontal_dist = np.linalg.norm(np.array([eye_landmarks[0].x, eye_landmarks[0].y]) - 
                                     np.array([eye_landmarks[3].x, eye_landmarks[3].y]))
    return vertical_dist / horizontal_dist

# Función para superponer la estrella en el ojo
def overlay_star(img, position, scale, mirror=False):
    star_resized = star_img.resize((scale, scale), Image.LANCZOS)
    star_np = np.array(star_resized)

    # Convertir la estrella de RGBA a BGR (sin el canal alfa)
    star_bgr = cv2.cvtColor(star_np, cv2.COLOR_RGBA2BGR)
    alpha = star_np[..., 3] / 255.0  # Canal alfa normalizado

    # Si se debe reflejar la estrella en el eje horizontal, reflejar también el canal alfa
    if mirror:
        star_bgr = cv2.flip(star_bgr, 1)
        alpha = np.flip(alpha, axis=1)  # Reflejar el canal alfa también

    # Posición en la imagen de destino
    x, y = int(position[0] - scale // 2), int(position[1] - scale // 2)

    # Asegúrate de que no nos salgamos de los límites de la imagen
    h, w = img.shape[:2]
    star_h, star_w = star_bgr.shape[:2]
    if x < 0 or x + star_w > w or y < 0 or y + star_h > h:
        return

    # Superponer la estrella sobre la imagen, utilizando el canal alfa para la mezcla
    for c in range(3):  # Canales de color BGR
        img[y:y+star_h, x:x+star_w, c] = (
            star_bgr[:, :, c] * alpha + 
            img[y:y+star_h, x:x+star_w, c] * (1 - alpha)
        )

# Iniciar captura de video
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    frame = cv2.flip(frame, 1)  # Voltear la imagen para mirarse en el espejo

    # Convertir solo a RGB para MediaPipe
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            left_eye_landmarks = [
                face_landmarks.landmark[i] for i in [362, 385, 387, 263, 373, 380]
            ]
            right_eye_landmarks = [
                face_landmarks.landmark[i] for i in [33, 160, 158, 133, 153, 144]
            ]

            left_ear = eye_aspect_ratio(left_eye_landmarks)
            right_ear = eye_aspect_ratio(right_eye_landmarks)

            # Determina si hay guiño en el ojo izquierdo o derecho
            if right_ear < 0.2 and left_ear > 0.3:
                # Guiño del ojo derecho detectado
                right_eye_center = (
                    int(right_eye_landmarks[0].x * frame.shape[1]),
                    int(right_eye_landmarks[0].y * frame.shape[0])
                )
                
                # Desplazamiento hacia la derecha
                right_eye_center = (right_eye_center[0] - 20, right_eye_center[1])

                overlay_star(frame, right_eye_center, 50, mirror=False)  # Estrella sin reflejo
                
            elif left_ear < 0.2 and right_ear > 0.3:
                # Guiño del ojo izquierdo detectado
                left_eye_center = (
                    int(left_eye_landmarks[0].x * frame.shape[1]),
                    int(left_eye_landmarks[0].y * frame.shape[0])
                )
                
                # Desplazamiento hacia la izquierda
                left_eye_center = (left_eye_center[0] + 60, left_eye_center[1])

                overlay_star(frame, left_eye_center, 50, mirror=True)  # Estrella reflejada

    # Mostrar la imagen directamente en BGR para OpenCV
    cv2.imshow("Wink Detection with Star Effect", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

### All Together

In [ ]:
# Configuración de MediaPipe
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True, static_image_mode=False, max_num_faces=1, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Cargar el GIF del arco iris y convertirlo a una lista de fotogramas
gif = Image.open("rainbow.gif")
gif_frames = [frame.copy() for frame in ImageSequence.Iterator(gif)]

# Cargar la imagen de la estrella y convertirla a RGBA
star_img = Image.open("sparkles.png").convert("RGBA")

def mouth_open(landmarks, threshold=0.05):
    """Calcula si la boca está abierta basado en la distancia entre puntos específicos."""
    top_lip = landmarks[13]  # Landmark del labio superior
    bottom_lip = landmarks[14]  # Landmark del labio inferior
    mouth_height = abs(bottom_lip.y - top_lip.y)
    return mouth_height > threshold

# Función para calcular la distancia vertical entre los párpados de un ojo
def eye_aspect_ratio(eye_landmarks):
    vertical_dist = np.linalg.norm(np.array([eye_landmarks[1].x, eye_landmarks[1].y]) - 
                                   np.array([eye_landmarks[5].x, eye_landmarks[5].y]))
    horizontal_dist = np.linalg.norm(np.array([eye_landmarks[0].x, eye_landmarks[0].y]) - 
                                     np.array([eye_landmarks[3].x, eye_landmarks[3].y]))
    return vertical_dist / horizontal_dist

# Función para superponer el GIF (arco iris) en la imagen
def gif_overlay(img, frame_position, scale_width):
    """Superpone el GIF escalado en la posición especificada de la imagen."""
    
    # Seleccionar el cuadro actual del GIF
    frame = gif_frames[int(cv2.getTickCount() % len(gif_frames)) % len(gif_frames)]
    
    # Redimensionar el GIF al tamaño adecuado
    frame = frame.resize((scale_width, int(scale_width * frame.size[1] / frame.size[0])))
    frame = frame.convert("RGBA")
    frame_np = np.array(frame)
    
    # Coordenadas y dimensiones
    mouth_center_x, mouth_center_y = frame_position
    gif_height, gif_width = frame_np.shape[:2]

    # Ajustar la posición de y para que el borde superior esté en el centro de la boca
    top_left_x = mouth_center_x - gif_width // 2
    top_left_y = mouth_center_y

    # Verificar límites de la imagen para no salirnos
    if top_left_y + gif_height > img.shape[0]:
        gif_height = img.shape[0] - top_left_y
        frame_np = frame_np[:gif_height, :, :]
    if top_left_x + gif_width > img.shape[1]:
        gif_width = img.shape[1] - top_left_x
        frame_np = frame_np[:, :gif_width, :]

    # Superponer el GIF en la imagen
    for c in range(3):  # Canales RGB
        img[top_left_y:top_left_y+gif_height, top_left_x:top_left_x+gif_width, c] = \
            frame_np[:gif_height, :gif_width, c] * (frame_np[:gif_height, :gif_width, 3] / 255.0) + \
            img[top_left_y:top_left_y+gif_height, top_left_x:top_left_x+gif_width, c] * \
            (1.0 - frame_np[:gif_height, :gif_width, 3] / 255.0)

# Función para superponer la estrella en el ojo (con posibilidad de reflejarla)
def overlay_star(img, position, scale, mirror=False):
    star_resized = star_img.resize((scale, scale), Image.LANCZOS)
    star_np = np.array(star_resized)

    # Convertir la estrella de RGBA a BGR (sin el canal alfa)
    star_bgr = cv2.cvtColor(star_np, cv2.COLOR_RGBA2BGR)
    alpha = star_np[..., 3] / 255.0  # Canal alfa normalizado

    # Si se debe reflejar la estrella en el eje horizontal, reflejar también el canal alfa
    if mirror:
        star_bgr = cv2.flip(star_bgr, 1)
        alpha = np.flip(alpha, axis=1)  # Reflejar el canal alfa también

    # Posición en la imagen de destino
    x, y = int(position[0] - scale // 2), int(position[1] - scale // 2)

    # Asegúrate de que no nos salgamos de los límites de la imagen
    h, w = img.shape[:2]
    star_h, star_w = star_bgr.shape[:2]
    if x < 0 or x + star_w > w or y < 0 or y + star_h > h:
        return

    # Superponer la estrella sobre la imagen, utilizando el canal alfa para la mezcla
    for c in range(3):  # Canales de color BGR
        img[y:y+star_h, x:x+star_w, c] = (
            star_bgr[:, :, c] * alpha + 
            img[y:y+star_h, x:x+star_w, c] * (1 - alpha)
        )

# Inicializar captura de video
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)  # Voltear la imagen para mirarse en el espejo

    # Convertir solo a RGB para MediaPipe
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            left_eye_landmarks = [
                face_landmarks.landmark[i] for i in [362, 385, 387, 263, 373, 380]
            ]
            right_eye_landmarks = [
                face_landmarks.landmark[i] for i in [33, 160, 158, 133, 153, 144]
            ]

            left_ear = eye_aspect_ratio(left_eye_landmarks)
            right_ear = eye_aspect_ratio(right_eye_landmarks)

            # Detectar si la boca está abierta
            mouth_top = face_landmarks.landmark[13]  # Labio superior
            mouth_bottom = face_landmarks.landmark[14]  # Labio inferior
            mouth_left = face_landmarks.landmark[78]  # Lado izquierdo de la boca
            mouth_right = face_landmarks.landmark[308]  # Lado derecho de la boca
            
            # Calcular posición y ancho de la boca
            mouth_center_x = int((mouth_top.x + mouth_bottom.x) / 2 * frame.shape[1])
            mouth_center_y = int((mouth_top.y + mouth_bottom.y) / 2 * frame.shape[0])
            mouth_width = int(abs(mouth_right.x - mouth_left.x) * frame.shape[1])

            # Verificar si la boca está abierta
            if mouth_open(face_landmarks.landmark):
                gif_overlay(frame, (mouth_center_x, mouth_center_y), mouth_width)

            # Determina si hay guiño en el ojo izquierdo o derecho
            if right_ear < 0.2 and left_ear > 0.3:
                # Guiño del ojo derecho detectado
                right_eye_center = (
                    int(right_eye_landmarks[0].x * frame.shape[1]),
                    int(right_eye_landmarks[0].y * frame.shape[0])
                )
                right_eye_center = (right_eye_center[0] - 20, right_eye_center[1])
                overlay_star(frame, right_eye_center, 50, mirror=False)  # Estrella sin reflejo
            elif left_ear < 0.2 and right_ear > 0.3:
                # Guiño del ojo izquierdo detectado
                left_eye_center = (
                    int(left_eye_landmarks[0].x * frame.shape[1]),
                    int(left_eye_landmarks[0].y * frame.shape[0])
                )
                left_eye_center = (left_eye_center[0] + 60, left_eye_center[1])
                overlay_star(frame, left_eye_center, 50, mirror=True)  # Estrella reflejada

    # Mostrar la imagen final con ambos efectos
    cv2.imshow("Wink and Rainbow Effect", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()